### Piesi

In [ ]:
import networkx as nx
#import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import os
from math import sin, cos, sqrt, atan2, radians
#from matplotlib_scalebar.scalebar import ScaleBar
from sklearn.metrics.pairwise import haversine_distances
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from scipy.optimize import curve_fit

#%matplotlib inline
#ox.config(log_console=True)
#ox.__version__

# Color blind library
plt.style.use('tableau-colorblind10')          
plt.style.library['tableau-colorblind10']
                

def getDistanceFromLatLonInM(lat1,lon1,lat2,lon2):
    """ Function that returns the distance in metres between 2 GPS locations in degrees (latitude and longitude).
    It is based in the Haversine formula (https://en.wikipedia.org/wiki/Haversine_formula) which takes into account the
    Earth's curvature. 
    
    Input:
        - 2 GPS coordinates: (latitude1,longitude1) of the first point and (latitude2,longitude2) of the second point. 
        
    Output:
        - Distance in metres between the two GPS locations.
    """
    
    R = 6371 # Radius of the earth in km
    dLat = radians(lat2-lat1)
    dLon = radians(lon2-lon1)
    rLat1 = radians(lat1)
    rLat2 = radians(lat2)
    a = sin(dLat/2) * sin(dLat/2) + cos(rLat1) * cos(rLat2) * sin(dLon/2) * sin(dLon/2) 
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c # Distance in km
    e= d*1000 #distance in m
    
    return e


In [ ]:
import datetime
compiled_data = pd.DataFrame()
all_files = glob.glob(os.path.join("*.csv"),root_dir="./processed and interpolated data") 
for file in all_files:   
    df = pd.read_csv("./processed and interpolated data/"+file)  # Read the csv file of each user
    compiled_data = compiled_data.append(df, ignore_index=True)
#compiled_data["timestamp"] = compiled_data.apply(lambda x: datetime.datetime.strptime(x.time,"%Y-%m-%d %H:%M:%S").timestamp())
#datetime.datetime.strptime(compiled_data["time"],"%Y-%m-%d %H:%M:%S").timestamp()
    
    
compiled_data.to_csv("compiled_data.csv", index=False)

In [ ]:
import geopy.distance
import datetime
movement = pd.read_csv("routes_multiple.csv")
lamps = pd.read_csv("AghLamps.csv")
movement.sort_values("time", inplace = True)
lampsOn = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])

In [ ]:
def calculate_destination_coordinates(lat, lon, distance, bearing):
    # Calculate the destination coordinates given the initial coordinates, distance, and bearing
    destination = geopy.distance.geodesic(kilometers=distance/1000).destination((lat, lon), bearing)
    return destination.latitude, destination.longitude


lamps['N'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 0)[0], axis=1)
lamps['E'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 90)[1], axis=1)
lamps['S'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 180)[0], axis=1)
lamps['W'] = lamps.apply(lambda row: calculate_destination_coordinates(row['Y'], row['X'], 20, 270)[1], axis=1)


In [ ]:
# Lamps NESW
lampsN = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])
lampsE = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])
lampsS = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])
lampsW = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])

def calcLamp(lamp, outputDF, sourceDf):
    if len(sourceDf)<1:
        return
    
    mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    prevtime = mintime
    for i in range (1,len(sourceDf)):
        current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
        if (((current - datetime.timedelta(seconds=10)) > prevtime) or i == len(sourceDf)-1):
            #save period and startover
            outputDF.loc[len(outputDF.index)] = [mintime, prevtime + datetime.timedelta(seconds=10), lamp.Y, lamp.X] 
            mintime = current
            prevtime = mintime
        else:
            prevtime = current


for l in range (1,len(lamps)):
    lamp = lamps.iloc[l]   
    calcLamp(lamp, lampsN, movement.loc[movement.apply(lambda x: (lamp.N>x['latitude'] and lamp.Y<x['latitude'] and lamp.E>x['longitude'] and lamp.W<x['longitude']),axis=1)])
    calcLamp(lamp, lampsE, movement.loc[movement.apply(lambda x: (lamp.N>x['latitude'] and lamp.S<x['latitude'] and lamp.E>x['longitude'] and lamp.X<x['longitude']),axis=1)])
    calcLamp(lamp, lampsS, movement.loc[movement.apply(lambda x: (lamp.Y>x['latitude'] and lamp.S<x['latitude'] and lamp.E>x['longitude'] and lamp.W<x['longitude']),axis=1)])
    calcLamp(lamp, lampsW, movement.loc[movement.apply(lambda x: (lamp.N>x['latitude'] and lamp.S<x['latitude'] and lamp.X>x['longitude'] and lamp.W<x['longitude']),axis=1)])
    
    lampsN.to_csv("lampN.csv", index=False)
    lampsE.to_csv("lampE.csv", index=False) 
    lampsS.to_csv("lampS.csv", index=False) 
    lampsW.to_csv("lampW.csv", index=False) 
    

In [ ]:
#Lamps Directional

lampsDir = pd.DataFrame(columns=['startTime','endTime','latitude','longitude','direction'])

import math

def calculate_heading(lat1, lon1, lat2, lon2):
    # Calculate the initial bearing between two points
    delta_lon = lon2 - lon1
    x = math.cos(math.radians(lat2)) * math.sin(math.radians(delta_lon))
    y = math.cos(math.radians(lat1)) * math.sin(math.radians(lat2)) - \
        math.sin(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.cos(math.radians(delta_lon))
    bearing = math.atan2(x, y)
    
    # Convert bearing to degrees and normalize to [0, 360) range
    heading = (math.degrees(bearing) + 360) % 360
    
    return heading


for l in range (1,len(lamps)):
    
    lamp = lamps.iloc[l]    
    sourceDf = movement.loc[movement.apply(lambda x: (lamp.N>x['latitude'] and lamp.S<x['latitude'] and lamp.E>x['longitude'] and lamp.W<x['longitude']),axis=1)]
    if len(sourceDf)<1:
        continue
    
    mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    prevtime = mintime
    for i in range (1,len(sourceDf)):
        current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
        lampsDir.loc[len(lampsDir.index)] = [current, current + datetime.timedelta(seconds=10), lamp.Y, lamp.X, calculate_heading(lamp.Y, lamp.X, sourceDf.iloc[i]['latitude'], sourceDf.iloc[i]['longitude'])]
            
lampsDir.to_csv("lampDir.csv", index=False)


In [ ]:
lampsDir

In [ ]:



for l in range (1,len(lamps)):
    
    lamp = lamps.iloc[l]    
    
    def lam(x):
        if (lamp.N>x['latitude'] and lamp.S<x['latitude'] and lamp.E>x['longitude'] and lamp.W<x['longitude']):
            return geopy.distance.distance((lamp.Y,lamp.X),(x['latitude'], x['longitude'])).m<20
        return False
    sourceDf = movement.loc[movement.apply(lam,axis=1)]
    if len(sourceDf)<1:
        continue
    
    mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
    prevtime = mintime
    for i in range (1,len(sourceDf)):
        current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
        if (((current - datetime.timedelta(seconds=10)) > prevtime) or i == len(sourceDf)-1):
            #save period and startover
            lampsOn.loc[len(lampsOn.index)] = [mintime, prevtime + datetime.timedelta(seconds=10), lamp.Y, lamp.X] 
            mintime = current
            prevtime = mintime
        else:
            prevtime = current
            
lampsOn.to_csv("lampOn.csv", index=False) 


In [ ]:
lampsOn

In [ ]:
lampsOn.to_csv("lampOn.csv", index=False) 

In [ ]:

lamp = (41.31587009,2.02229937)

#lam = lambda x: geopy.distance.distance((41.31587009,2.02229937),(x.latitude, x.longtitude))
def lam(x):
    return geopy.distance.distance((41.31587009,2.02229937),(x['latitude'], x['longitude'])).m<20

rslt_df = compiled_data.loc[compiled_data.apply(lam,axis=1)]
rslt_df.to_csv("lamp1.csv", index=False)


        
#compiled_data["lDist"] = compiled_data.apply(lam,axis=1)
# 225082.10, 5059301.28

In [ ]:
lampsOn = pd.DataFrame(columns=['startTime','endTime','latitude','longitude'])

sourceDf = rslt_df.sort_values("time")

mintime = datetime.datetime.strptime(sourceDf.iloc[0].time, "%Y-%m-%d %H:%M:%S")
prevtime = mintime
for i in range (1,len(sourceDf)):
    current = datetime.datetime.strptime(sourceDf.iloc[i].time, "%Y-%m-%d %H:%M:%S")
    print(f"{current}    :     {prevtime}\n{((current - datetime.timedelta(seconds=10)) > prevtime)}\n")
    if ((current - datetime.timedelta(seconds=10)) > prevtime):
        #save period and startover
        lampsOn.loc[len(lampsOn.index)] = [mintime, prevtime + datetime.timedelta(seconds=10), lamp[0], lamp[1]] 
        mintime = current
        prevtime = mintime
    else:
        prevtime = current
lampsOn.to_csv("lampOn.csv", index=False)

### Awarie Lamp

In [ ]:
# Zakładamy że Lampy co jakiś czas (minuta) pytają sąsiadów czy wszystko w porządku, jeśli sąsiad nie odpowiada przez kilka wiadomości podrząd, zakładamy że się zepsuł i raportujemy usterkę
# wybierz 2 lampy do awari i wylosuj kiedy się zepsują
# Lampy w promienu 50 metrów zauważają awarię po losowym okresie czasu (0-60s + 2 min) i zgłaszają ją do rozdzielni

# TODO : Usunąć zaświecane uszkodzonej lampy

import random
from  datetime import timedelta

lampsFail = pd.DataFrame(columns=['startTime','endTime','longitude','latitude'])
lampsAlert = pd.DataFrame(columns=['startTime','endTime','longitude','latitude'])



startDateTime = datetime.datetime.strptime(movement['time'].min(), "%Y-%m-%d %H:%M:%S")
endDateTime = datetime.datetime.strptime(movement['time'].max(), "%Y-%m-%d %H:%M:%S")
timeDelta = (endDateTime - startDateTime).total_seconds()

for i in range(2):
    victim = lamps.sample().iloc[0]
    timeOfDeath = startDateTime + timedelta(seconds=random.uniform(0, timeDelta))
    lampsFail.loc[len(lampsFail.index)] = [timeOfDeath, endDateTime, victim.X, victim.Y] 

    def lam(x):
        return geopy.distance.distance((victim.Y,victim.X),(x.Y, x.X)).m<50
    Neighbours = lamps.loc[lamps.apply(lam,axis=1)]
    #Neighbours = lamps[geopy.distance.distance((victim.Y,victim.X),(lamps['Y'], lamps['X'])).m<50]
    for index, n in Neighbours.iterrows():
        alertTime = timeOfDeath + timedelta(seconds=random.uniform(120, 180))
        if (victim.Y != n.Y and victim.X != n.X):
            lampsAlert.loc[len(lampsAlert.index)] = [alertTime, endDateTime, n.X, n.Y]


lampsFail.to_csv("lampFail.csv", index=False) 
lampsAlert.to_csv("lampAlert.csv", index=False) 

In [ ]:
for index, row in Neighbours.iterrows():
    print(row.X)

In [ ]:
#sourceDf.iloc[200:250]
movement[0:50]

In [ ]:
type(startDateTime)

In [ ]:
startDateTime

In [ ]:
df